In [17]:
import pandas as pd
import numpy as np
import mysql.connector
import altair as alt
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
from scipy.stats import ttest_rel
from scipy.stats import shapiro, levene, ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from urllib.parse import quote_plus
from sqlalchemy_utils import create_database, database_exists
from sqlalchemy import MetaData
import pymysql
from sqlalchemy.orm import Session
from sqlalchemy import Table, Column, Integer, BIGINT, String, BigInteger, Boolean, Float, Unicode
from sqlalchemy import ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import LONGTEXT
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [18]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="3901",
    database="laptop_zoomit_new",
    auth_plugin='mysql_native_password'
)

In [19]:
query = """
SELECT product.title, product.model, product.price, storage.size, storage.type, gpu.type, gpu_ram.size, manufacturer.name, screen_size.size, cpu.name, ram.name
FROM product
         left JOIN manufacturer on product.manufacturer_id = manufacturer.id
         left join screen_size on product.screen_size_id = screen_size.id
         left join cpu on product.cpu_id = cpu.id
         left join ram on product.ram_id = ram.id
         left join product_storage on product.id = product_storage.id_product
         left join storage on product_storage.id_storage = storage.id
         left join gpu on product.gpu_id = gpu.id
         left join gpu_ram on product.gpu_ram_id = gpu_ram.id
"""
mycursor = mydb.cursor()
mycursor.execute(query)
products = mycursor.fetchall()
df_products = pd.DataFrame(
    products,
    columns=['title', 'model', 'price', 'storage_size', 'storage_type', 'gpu_type', 'gpu_size', 'manufacturer', 'screen_size', 'cpu', 'ram'])
df_products

,title,model,price,storage_size,storage_type,gpu_type,gpu_size,manufacturer,screen_size,cpu,ram
0,لپ تاپ سرفیس لپ تاپ گو مایکروسافت - Core i5-10...,None,32988000,256.0,SSD,None,None,None,12.4,اینتل Core i5-1035G1,8 گیگابایت
1,لپ تاپ Aspire 3 A315-59G-35XQ ایسر - Core i3-1...,None,23500000,256.0,SSD,انویدیا GeForce MX550,2 گیگابایت از نوع GDDR6,None,15.6,اینتل Core i3-1215U,4 گیگابایت
2,لپ تاپ TUF Dash F15 FX517ZR ایسوس - Core i7-12...,TUF Dash F15 FX517ZR,66199000,1.0,SSD,انویدیا GeForce RTX 3070,8 گیگابایت از نوع GDDR6,ASUS,15.6,اینتل Core i7-12650H,32 گیگابایت
3,لپ تاپ ROG Strix G17 G713RC ایسوس - Ryzen 7 68...,ROG Strix G17 G713RC 2022,53999000,1.0,PCIe NVMe,انویدیا GeForce RTX 3050,4 گیگابایت از نوع GDDR6 و 128 بیت,ASUS,17.3,AMD Ryzen 7 6800H,16 گیگابایت
4,لپ تاپ V15 G2 لنوو - Core i5-1135G7 MX350 8GB ...,None,25795000,1.0,HDD,انویدیا GeForce MX350,2 گیگابایت,None,15.6,اینتل Core i5-1135G7,8 گیگابایت
...,...,...,...,...,...,...,...,...,...,...,...
1295,لپ تاپ VivoBook X515EP ایسوس - Core i3-1115G4 ...,VivoBook X515EP,20800000,512.0,SSD,انویدیا GeForce MX330,2 گیگابایت از نوع GDDR5,ASUS,15.6,اینتل Core i3-1115G4,12 گیگابایت
1296,لپ تاپ V15 G3 لنوو - Core i3-1215U UHD 20GB 15...,None,20900000,1.0,HDD,None,None,None,15.6,اینتل Core i3-1215U,20 گیگابایت
1297,لپ تاپ V15 G3 لنوو - Core i3-1215U UHD 20GB 15...,None,20900000,512.0,SSD,None,None,None,15.6,اینتل Core i3-1215U,20 گیگابایت
1298,لپ تاپ V15 G3 لنوو - Core i3-1215U UHD 20GB 11...,None,20600000,1.0,HDD,None,None,None,15.6,اینتل Core i3-1215U,20 گیگابایت


In [20]:
columns_to_drop = ['title', 'model']
df_products = df_products.drop(columns=columns_to_drop)
df_products = df_products.drop_duplicates()
df_products = df_products.apply(lambda x: x.fillna(x.mode()[0]))
df_products

,price,storage_size,storage_type,gpu_type,gpu_size,manufacturer,screen_size,cpu,ram
0,32988000,256.0,SSD,انویدیا GeForce MX350,2 گیگابایت,ASUS,12.4,اینتل Core i5-1035G1,8 گیگابایت
1,23500000,256.0,SSD,انویدیا GeForce MX550,2 گیگابایت از نوع GDDR6,ASUS,15.6,اینتل Core i3-1215U,4 گیگابایت
2,66199000,1.0,SSD,انویدیا GeForce RTX 3070,8 گیگابایت از نوع GDDR6,ASUS,15.6,اینتل Core i7-12650H,32 گیگابایت
3,53999000,1.0,PCIe NVMe,انویدیا GeForce RTX 3050,4 گیگابایت از نوع GDDR6 و 128 بیت,ASUS,17.3,AMD Ryzen 7 6800H,16 گیگابایت
4,25795000,1.0,HDD,انویدیا GeForce MX350,2 گیگابایت,ASUS,15.6,اینتل Core i5-1135G7,8 گیگابایت
...,...,...,...,...,...,...,...,...,...
1295,20800000,512.0,SSD,انویدیا GeForce MX330,2 گیگابایت از نوع GDDR5,ASUS,15.6,اینتل Core i3-1115G4,12 گیگابایت
1296,20900000,1.0,HDD,انویدیا GeForce MX350,2 گیگابایت,ASUS,15.6,اینتل Core i3-1215U,20 گیگابایت
1297,20900000,512.0,SSD,انویدیا GeForce MX350,2 گیگابایت,ASUS,15.6,اینتل Core i3-1215U,20 گیگابایت
1298,20600000,1.0,HDD,انویدیا GeForce MX350,2 گیگابایت,ASUS,15.6,اینتل Core i3-1215U,20 گیگابایت


In [21]:
df_products['cpu'] = df_products['cpu'].astype('category').cat.codes
df_products['storage_type'] = df_products['storage_type'].astype('category').cat.codes
df_products['manufacturer'] = df_products['manufacturer'].astype('category').cat.codes
df_products['gpu_type'] = df_products['gpu_type'].astype('category').cat.codes
df_products['gpu_size'] = df_products['gpu_size'].astype('category').cat.codes
df_products['ram'] = df_products['ram'].str.replace('گیگابایت', '').str.strip()
df_products

,price,storage_size,storage_type,gpu_type,gpu_size,manufacturer,screen_size,cpu,ram
0,32988000,256.0,4,10,6,0,12.4,49,8
1,23500000,256.0,4,12,10,0,15.6,44,4
2,66199000,1.0,4,19,33,0,15.6,85,32
3,53999000,1.0,3,16,18,0,17.3,20,16
4,25795000,1.0,0,10,6,0,15.6,56,8
...,...,...,...,...,...,...,...,...,...
1295,20800000,512.0,4,9,8,0,15.6,42,12
1296,20900000,1.0,0,10,6,0,15.6,44,20
1297,20900000,512.0,4,10,6,0,15.6,44,20
1298,20600000,1.0,0,10,6,0,15.6,44,20


In [22]:
X = df_products.drop('price', axis=1)
y = df_products['price']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [44]:
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("R2 Score:", r2)
print("MSE:", mse)
print("MAE:", mae)
print("RMSE:", rmse)


R2 Score: 0.8506588942825687
MSE: 110433224701186.73
MAE: 6117580.174963664
RMSE: 10508721.363761947


In [45]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("R2 Score:", r2)
print("MSE:", mse)
print("MAE:", mae)
print("RMSE:", rmse)

R2 Score: 0.5571389500960564
MSE: 327482333818941.5
MAE: 11616361.309963614
RMSE: 18096472.966270015


In [46]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("R2 Score:", r2)
print("MSE:", mse)
print("MAE:", mae)
print("RMSE:", rmse)

R2 Score: 0.8254618770544484
MSE: 129065655819097.6
MAE: 6287646.026355953
RMSE: 11360706.660199339


In [47]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("R2 Score:", r2)
print("MSE:", mse)
print("MAE:", mae)
print("RMSE:", rmse)

R2 Score: 0.5600118608889859
